From https://lernapparat.de/jit-optimization-intro/

In [2]:
import torch
from torch import Tensor

In [3]:
def fn(x):
    for i in range(x.dim()):
        x = x * x
    return x

script_fn = torch.jit.script(fn)
trace_fn = torch.jit.trace(fn, [torch.randn(5, 5)])

print(script_fn.code)

print(trace_fn.code)


def fn(x: Tensor) -> Tensor:
  x0 = x
  for i in range(torch.dim(x)):
    x0 = torch.mul(x0, x0)
  return x0

def fn(x: Tensor) -> Tensor:
  x0 = torch.mul(x, x)
  return torch.mul(x0, x0)



In [4]:
@torch.jit.script
def conv1(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    (na,) = a.size()
    (nb,) = b.size()
    out = torch.empty(na - nb + 1)
    for ia in range(na - nb + 1):
        out[ia] = sum(torch.tensor([a[ia + ib] * b[ib] for ib in range(na)]))
    return out

print(conv1.code)

def conv1(a: Tensor,
    b: Tensor) -> Tensor:
  na, = torch.size(a)
  nb, = torch.size(b)
  out = torch.empty([torch.add(torch.sub(na, nb), 1)])
  for ia in range(torch.add(torch.sub(na, nb), 1)):
    _0 = annotate(List[Tensor], [])
    for ib in range(na):
      _1 = torch.select(a, 0, torch.add(ia, ib))
      _2 = torch.mul(_1, torch.select(b, 0, ib))
      _3 = torch.append(_0, _2)
    _4 = torch.sum(torch.tensor(_0))
    _5 = torch.copy_(torch.select(out, 0, ia), _4)
  return out



In [13]:
def ratio_iou(x1, y1, w1, h1, x2, y2, w2, h2):
    xi = torch.max(x1, x2)                                  # Intersection left
    yi = torch.max(y1, y2)                                  # Intersection top
    wi = torch.clamp(torch.min(x1+w1, x2+w2) - xi, min=0.)  # Intersection width
    hi = torch.clamp(torch.min(y1+h1, y2+h2) - yi, min=0.)  # Intersection height
    area_i = wi * hi                                        # Area Intersection
    area_u = w1 * h1 + w2 * h2 - wi * hi                    # Area Union
    return area_i / torch.clamp(area_u, min=1e-5)           # Intersection over Union

import torch.jit

with torch.jit.optimized_execution(True):
  ratio_iou_scripted = torch.jit.script(ratio_iou)

  print(ratio_iou_scripted.code)

def ratio_iou(x1: Tensor,
    y1: Tensor,
    w1: Tensor,
    h1: Tensor,
    x2: Tensor,
    y2: Tensor,
    w2: Tensor,
    h2: Tensor) -> Tensor:
  xi = torch.max(x1, x2)
  yi = torch.max(y1, y2)
  _0 = torch.min(torch.add(x1, w1), torch.add(x2, w2))
  wi = torch.clamp(torch.sub(_0, xi), 0.)
  _1 = torch.min(torch.add(y1, h1), torch.add(y2, h2))
  hi = torch.clamp(torch.sub(_1, yi), 0.)
  area_i = torch.mul(wi, hi)
  _2 = torch.add(torch.mul(w1, h1), torch.mul(w2, h2))
  area_u = torch.sub(_2, torch.mul(wi, hi))
  _3 = torch.clamp(area_u, 1.0000000000000001e-05)
  return torch.div(area_i, _3)



In [25]:
# we make a scripted function
ratio_iou_scripted = torch.jit.script(ratio_iou)

x1, y1, w1, h1, x2, y2, w2, h2 = torch.randn(8, 100, 1000, device='cuda', requires_grad=True).exp()

ratio_iou_scripted(x1, y1, w1, h1, x2, y2, w2, h2)

print(ratio_iou_scripted.graph_for(x1, y1, w1, h1, x2, y2, w2, h2))



graph(%x1.1 : Tensor,
      %y1.1 : Tensor,
      %w1.1 : Tensor,
      %h1.1 : Tensor,
      %x2.1 : Tensor,
      %y2.1 : Tensor,
      %w2.1 : Tensor,
      %h2.1 : Tensor):
  %8 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %9 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %10 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %11 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %12 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %13 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %14 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %15 : Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), %16 : bool = prim::TypeCheck[types=[Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), Float(100, 1000, strides=[1000, 1], requires_grad=0, device=cuda:0), Float(100, 1000, strides=[1000, 1], requires_grad